In [ ]:
import pandas as pd
import requests
import os
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict, Counter

In [4]:
# Verified necessary libraries
import numpy as np
import pandas as pd
import gzip
import json

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/book_recommendation')
os.listdir('./data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['2000users_120interactions_100000booksample_bert_embeddings_fantasy.json.gz',
 'goodreads_interactions_2k_users_fantasy_paranormal.json.gz',
 'goodreads_interactions_20k_users_book_ids.csv',
 'goodreads_books_fantasy_paranormal.json.gz',
 'smaller_books_100000.json.gz',
 'goodreads_interactions_fantasy_paranormal.json.gz']

In [3]:
DIR = './data'

In [11]:
def load_interactions(file_name, max_user_cnt=20000, max_user_with_min_interaction_cnt=2000, min_interactions=120):
    count = 0
    data = []
    user_interactions = dict()
    users_with_min_interaction = set()

    # Use gzip.open in text mode
    with gzip.open(file_name, 'rt', encoding='utf-8') as fin:
        for l in fin:
            d = json.loads(l)
            if d.get('is_read', False):  # Only keep read interactions
                user_id = d["user_id"]
                if user_id not in user_interactions and len(user_interactions) < max_user_cnt:
                    user_interactions[user_id] = 1
                else:
                    user_interactions[user_id] += 1
                    if user_interactions[user_id] > min_interactions:
                        users_with_min_interaction.add(user_id)
                if user_id in user_interactions:
                    data.append(d)
                if len(users_with_min_interaction) >= max_user_with_min_interaction_cnt:
                    break

    filtered_data = [entry for entry in data if entry["user_id"] in users_with_min_interaction]
    return filtered_data


In [12]:
interactions = load_interactions(os.path.join(DIR, 'goodreads_interactions_fantasy_paranormal.json.gz'))

In [9]:
print(interactions[:1])

[{'user_id': 'f8a89075dc6de14857561522e729f82c', 'book_id': '27409149', 'review_id': '5f9ae8d613787ccbfab1db5eb05157db', 'is_read': True, 'rating': 4, 'review_text_incomplete': '', 'date_added': 'Wed Jul 27 09:40:11 -0700 2016', 'date_updated': 'Wed Jul 27 09:41:36 -0700 2016', 'read_at': 'Wed Jul 27 09:41:36 -0700 2016', 'started_at': 'Wed Jul 27 00:00:00 -0700 2016'}]


In [13]:
unique_user_count = len(set(item["user_id"] for item in interactions))
unique_book_count = len(set(item["book_id"] for item in interactions))
print("user count: ", unique_user_count, " book count: ", unique_book_count)

user count:  2000  book count:  81867


In [16]:
df = pd.DataFrame(interactions)
user_book_counts = df.groupby('user_id').size()

bucketed_counts = pd.cut(user_book_counts, bins=[0, 120, 240, 360, 500, 1000,  float('inf')])

distribution = bucketed_counts.value_counts(normalize=True) * 100  # In percentage
print(distribution)

(120.0, 240.0]     65.85
(240.0, 360.0]     18.80
(360.0, 500.0]      8.05
(500.0, 1000.0]     6.20
(1000.0, inf]       1.10
(0.0, 120.0]        0.00
Name: proportion, dtype: float64


In [17]:
unique_book_ids = {item["book_id"] for item in interactions}
df = pd.DataFrame(unique_book_ids, columns=["book_id"])
df.to_csv(os.path.join(DIR, "goodreads_interactions_2k_users_book_ids.csv"), index=False)

In [18]:
df = pd.DataFrame(interactions)
df.to_json(os.path.join(DIR, "goodreads_interactions_2k_users.json.gz"), orient="records", compression="gzip")